In [1]:
import pandas as pd
import numpy as np

In [36]:
# Gather all csv files with required features

df_pit_stop = pd.read_csv('/Users/tarun/Documents/Projects/GitHub/Formula-1/our-data/2020/races/01-austria/pit-stops.csv')
df_reace_results = pd.read_csv('/Users/tarun/Documents/Projects/GitHub/Formula-1/our-data/2020/races/01-austria/race-results.csv')
df_fastest_laps = pd.read_csv('/Users/tarun/Documents/Projects/GitHub/Formula-1/our-data/2020/races/01-austria/fastest-laps.csv')
df_driver_standings = pd.read_csv('/Users/tarun/Documents/Projects/GitHub/Formula-1/our-data/2020/races/01-austria/driver-standings.csv')

print(df_pit_stop.columns, "\n", df_driver_standings.columns, "\n", df_fastest_laps.columns, "\n", df_reace_results.columns)

Index(['time', 'constructorId', 'engineManufacturerId', 'stop', 'driverId',
       'position', 'tyreManufacturerId', 'lap', 'driverNumber'],
      dtype='object') 
 Index(['driverId', 'position', 'points'], dtype='object') 
 Index(['time', 'constructorId', 'engineManufacturerId', 'interval', 'gap',
       'driverId', 'position', 'tyreManufacturerId', 'lap', 'driverNumber'],
      dtype='object') 
 Index(['time', 'constructorId', 'gridPosition', 'engineManufacturerId',
       'timePenalty', 'interval', 'laps', 'gap', 'driverId', 'reasonRetired',
       'position', 'points', 'tyreManufacturerId', 'driverNumber'],
      dtype='object')


In [100]:
df_pit_stop.rename(columns={'position': 'pit_stop_lap_position'}, inplace=True)
df_pit_stop.rename(columns={'lap': 'pit_stop_lap'}, inplace=True)
df_pit_stop = df_pit_stop[['stop', 'driverId',  'constructorId', 'pit_stop_lap_position', 'pit_stop_lap']]


df_pit_stop



,stop,driverId,constructorId,pit_stop_lap_position,pit_stop_lap
0,1,romain-grosjean,haas,1,20
1,1,kimi-raikkonen,alfa-romeo,2,24
2,1,antonio-giovinazzi,alfa-romeo,3,25
3,1,george-russell,williams,4,25
4,1,valtteri-bottas,mercedes,5,26
5,1,lewis-hamilton,mercedes,6,26
6,1,alexander-albon,red-bull,7,26
7,1,lando-norris,mclaren,8,26
8,1,sergio-perez,racing-point,9,26
9,1,charles-leclerc,ferrari,10,26


In [191]:
max_stops = df_pit_stop['stop'].max()

merged_dfs = {}
for stop in range(0, max_stops):
    merged_dfs[stop] = df_pit_stop[df_pit_stop['stop'] == stop+1]
    merged_dfs[stop].rename(columns={'pit_stop_lap': 'pit_stop_'+str(stop+1)+'_lap'}, inplace=True)
    merged_dfs[stop].rename(columns={'pit_stop_lap_position': 'pit_stop_'+str(stop+1)+'_lap_position'}, inplace=True)





# for stop in range(0, len(merged_dfs)):
#     merged_dfs[stop]['pit_stop_'+str(stop+1)] = merged_dfs[stop]['pit_stop_lap']

# for stop in range (1, len(merged_dfs)):
#     merged_dfs[stop] = merged_dfs[stop][['driverId', 'pit_stop_'+str(stop+1)]]

# final_df = merged_dfs[0][['driverId']]
# for stop in range (0, len(merged_dfs)):
#     if stop != len(merged_dfs)-1:
#         merged_df_new = pd.merge( merged_dfs[stop],  merged_dfs[stop+1], on='driverId', how='inner')
#         final_df = pd.merge( final_df,  merged_df_new, on='driverId', how='inner')


# # merged_df_new = pd.merge( merged_dfs[0],  merged_dfs[1], on='driverId', how='inner')

# final_df



/var/folders/ds/m5y56fb13zq237kh9jr6plw40000gn/T/ipykernel_35430/2609057431.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_dfs[stop].rename(columns={'pit_stop_lap': 'pit_stop_'+str(stop+1)+'_lap'}, inplace=True)
/var/folders/ds/m5y56fb13zq237kh9jr6plw40000gn/T/ipykernel_35430/2609057431.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_dfs[stop].rename(columns={'pit_stop_lap_position': 'pit_stop_'+str(stop+1)+'_lap_position'}, inplace=True)


In [197]:
def merge_fun(merged_dfs):

    # merged_1 = merged_dfs[0]
    # for i in range (1, len(merged_dfs)):
    #     merge1 = pd.merge(merged_1, merged_dfs[i], on = ["driverId", "constructorId"], how = "left")


    merged_1 = merged_dfs[0]
    for i in range (1, len(merged_dfs)):
        merged_1 = pd.merge(merged_1, merged_dfs[i], on = ["driverId"], how = "left")

        

    # df1 = merged_dfs[0]
    # df2 = merged_dfs[1]
    # df3 = merged_dfs[2]

    # print(df1.shape, df2.shape, df3.shape)

    # merge1 = pd.merge(df1, df2, on = ["driverId", "constructorId"], how = "left")
    # merge2 = pd.merge(merge1, df3, on = ["driverId", "constructorId"], how = "left")


    return merged_1[['driverId', 'pit_stop_1_lap', 'pit_stop_2_lap', 'pit_stop_3_lap']]


In [198]:
m_df = merge_fun(merged_dfs)

In [199]:
m_df

,driverId,pit_stop_1_lap,pit_stop_2_lap,pit_stop_3_lap
0,romain-grosjean,20,NaN,NaN
1,kimi-raikkonen,24,51.0,NaN
2,antonio-giovinazzi,25,55.0,56.0
3,george-russell,25,NaN,NaN
4,valtteri-bottas,26,56.0,NaN
5,lewis-hamilton,26,56.0,NaN
6,alexander-albon,26,51.0,56.0
7,lando-norris,26,51.0,56.0
8,sergio-perez,26,56.0,NaN
9,charles-leclerc,26,51.0,56.0


In [34]:
max_stops = df_pit_stop['stop'].max()

merged_dfs = {}
for stop in range(0, max_stops):
    merged_dfs[stop] = df_pit_stop[df_pit_stop['stop'] == stop+1]

merged_dfs[2]




,stop,driverId,constructorId,pit_stop_lap_position,pit_stop_lap,pit_stop_1,pit_stop_2,pit_stop_3
28,3,alexander-albon,red-bull,29,56,0,0,56
29,3,lando-norris,mclaren,30,56,0,0,56
30,3,charles-leclerc,ferrari,31,56,0,0,56
32,3,carlos-sainz-jr,mclaren,33,56,0,0,56
34,3,daniil-kvyat,alphatauri,35,56,0,0,56
35,3,sebastian-vettel,ferrari,36,56,0,0,56
36,3,nicholas-latifi,williams,37,56,0,0,56
37,3,antonio-giovinazzi,alfa-romeo,38,56,0,0,56


In [41]:
df_fastest_laps = df_fastest_laps[['driverId', 'position']]
df_fastest_laps.head()

,driverId,position
0,lando-norris,1
1,valtteri-bottas,2
2,lewis-hamilton,3
3,charles-leclerc,4
4,carlos-sainz-jr,5


In [42]:
df_fastest_laps["fastets_lap"] = df_fastest_laps['position'].apply(lambda x: 1 if x == 1 else 0)
del df_fastest_laps['position']
df_fastest_laps.head()

,driverId,fastets_lap
0,lando-norris,1
1,valtteri-bottas,0
2,lewis-hamilton,0
3,charles-leclerc,0
4,carlos-sainz-jr,0


In [43]:
df_reace_results.head()

,time,constructorId,gridPosition,engineManufacturerId,timePenalty,interval,laps,gap,driverId,reasonRetired,position,points,tyreManufacturerId,driverNumber
0,1:30:55.739,mercedes,1,mercedes,NaN,NaN,71,NaN,valtteri-bottas,NaN,1,25.0,pirelli,77
1,1:30:58.439,ferrari,7,ferrari,NaN,2.700,71,2.700,charles-leclerc,NaN,2,18.0,pirelli,16
2,1:31:01.230,mclaren,3,renault,NaN,2.791,71,5.491,lando-norris,NaN,3,16.0,pirelli,4
3,1:31:01.428,mercedes,5,mercedes,5.0,0.198,71,5.689,lewis-hamilton,NaN,4,12.0,pirelli,44
4,1:31:04.642,mclaren,8,renault,NaN,3.214,71,8.903,carlos-sainz-jr,NaN,5,10.0,pirelli,55


In [44]:
df_reace_results = df_reace_results[['gridPosition','driverId']]
df_reace_results.head()

,gridPosition,driverId
0,1,valtteri-bottas
1,7,charles-leclerc
2,3,lando-norris
3,5,lewis-hamilton
4,8,carlos-sainz-jr


In [ ]:
df_mer_1 = pd.merge(df_reace_results, df_fastest_laps, how='inner', on='driverId')
df_mer_2 = pd.merge(df_mer_1, df_pit_stop, how='inner', on='driverId')
df_mer_3 = pd.merge(df_mer_2, df_driver_standings, how='inner', on='driverId')

In [46]:
df_mer_3.head()

,gridPosition,driverId,fastets_lap,stop,constructorId,pit_stop_lap_position,pit_stop_lap,position,points
0,1,valtteri-bottas,0,1,mercedes,5,26,1,25
1,1,valtteri-bottas,0,2,mercedes,26,56,1,25
2,7,charles-leclerc,0,1,ferrari,10,26,2,18
3,7,charles-leclerc,0,2,ferrari,19,51,2,18
4,7,charles-leclerc,0,3,ferrari,31,56,2,18


In [47]:
df_mer_3.rename(columns={'position': 'final_position'}, inplace=True)
df_mer_3.head()

,gridPosition,driverId,fastets_lap,stop,constructorId,pit_stop_lap_position,pit_stop_lap,final_position,points
0,1,valtteri-bottas,0,1,mercedes,5,26,1,25
1,1,valtteri-bottas,0,2,mercedes,26,56,1,25
2,7,charles-leclerc,0,1,ferrari,10,26,2,18
3,7,charles-leclerc,0,2,ferrari,19,51,2,18
4,7,charles-leclerc,0,3,ferrari,31,56,2,18


In [12]:
pd.merge??


Signature:
pd.merge(
    left: 'DataFrame | Series',
    right: 'DataFrame | Series',
    how: 'MergeHow' = 'inner',
    on: 'IndexLabel | AnyArrayLike | None' = None,
    left_on: 'IndexLabel | AnyArrayLike | None' = None,
    right_on: 'IndexLabel | AnyArrayLike | None' = None,
    left_index: 'bool' = False,
    right_index: 'bool' = False,
    sort: 'bool' = False,
    suffixes: 'Suffixes' = ('_x', '_y'),
    copy: 'bool | None' = None,
    indicator: 'str | bool' = False,
    validate: 'str | None' = None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

A named Series object is treated as a DataFrame with a single named column.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.
When performing a cross merge, no column specifications to merge on are
allowed.

.. warning: